# Aula 7 - Exercícios - Pandas Parte 3 (Merge e Join)

In [2]:
!pip install --upgrade ipynb_autograde > /dev/null

In [ ]:
%env  COURSE=mba_enap
from ipynb_autograde.main import init_log, validate
init_log()
email_result=!gcloud config get-value account
if not email_result or 'unset' in email_result[0]:
  !gcloud auth login

In [ ]:
# Importe a biblioteca pandas
import pandas as pd

### 7.1 Faça um prompt para criar uma função python que integre os dados de PIB de 2017 (Exercício 6.7) e da população (Exercício 6.3) carregados no notebook da aula anterior. Como sugestão, reaproveite o prompt desses dois exercícios e faça um prompt que carregue novamente os arquivos e na sequencia instrua o modelo a integra-los.

#### Entradas: ```https://github.com/alexlopespereira/mba_enap/raw/main/data/originais/pib/pib_municipios.xlsx``` e ```https://github.com/alexlopespereira/mba_enap/raw/main/data/originais/populacao/POP2024_20241101.xls```

#### Resultado esperado:



In [ ]:
prompt71 = """Escreva uma função Python utilizando pandas para carregar um arquivo Excel de uma URL passada como 1o argumento. O código deve: Ler a planilha chamada 'MUNICÍPIOS', ignorando a primeira linha e interpretando as colunas 'COD. UF' e 'COD. MUNIC' como string; Remover a última coluna do DataFrame; Alterar o nome duas últimas colunas para 'MUNICIPIO' e 'POPULACAO', respectivamente; Criar uma nova coluna chamada 'cod_ibge7', concatenando 'COD. UF' e 'COD. MUNIC'; Excluir as últimas 40 linhas do DataFrame. Nomeie esse dataframe como df_pop. A função tem num segundo argumento o caminho de um arquivo Excel carregue o arquivo ignorando a última linha, forçando a coluna CodIBGE como string, e nomeando esse dataframe como df_pib. Faça um merge entre esses dois dataframes, considerando que a chave comum no df_pop se chama cod_ibge7 e no df_pib se chama CodIBGE."""

In [ ]:
validate(prompt71, "A7-E1")

In [6]:
# prompt: carregue os dados desse xlsx num pandas dataframe https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/populacao/estimativa_dou_2017.xlsx

import pandas as pd

def load_dataframe_from_xlsx(url):
    df = pd.read_excel(url)
    return df

url = 'https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/populacao/estimativa_dou_2017.xlsx'
df = load_dataframe_from_xlsx(url)

df.head()

,uf,cod_uf,cod_munic,cod_ibge,municipio,pop2017
0,RO,11,15,1100015,Alta Floresta D'Oeste,25437
1,RO,11,23,1100023,Ariquemes,107345
2,RO,11,31,1100031,Cabixi,6224
3,RO,11,49,1100049,Cacoal,88507
4,RO,11,56,1100056,Cerejeiras,17934


### 7.2 Escreva uma função para calcular o PIB percapita  de 2017 de cada município do Brasil, considerando o dataframe resultante do exercício anterior.
#### Nomeie a coluna do PIB Percapita com 'pib_percapita' e arredonde o resultado para uma casa decimal usando a função ```round```.
#### Dica: Use a função ```round``` dentro de uma função ```apply```.

In [ ]:
def pib_percapita(pib_pop):
    """retorne um dataframe contendo o PIB percapita dos municípios.
    """
    ### BEGIN SOLUTION
    pib_pop['pib_percapita'] = pib_pop['2017']/pib_pop['pop2017']
    pib_pop['pib_percapita'] = pib_pop['pib_percapita'].apply(lambda x: round(x, 1))
    return pib_pop
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui



In [ ]:
# Validação
# Considerando a função merge_pop_pib do exercicio 7.1
df_pib_pop = merge_pop_pib(path_pop, path_pib)

entradas = [[df_pib_pop]]
saidas = [19.6]
validate(pib_percapita, entradas, lambda x: x.loc[0,'pib_percapita'], saidas, "7.2", False)

### 7.3 Escreva uma função para retornar os registros dos 10 maiores PIBs percapita do brasil (de 2017) a partir do dataframe do exercício anterior.
#### Escreva uma função genérica que receba como parâmetro um dataframe e o nome da coluna de ordenação.
#### Dica 1: Use o método sort_values para ordenar o dataframe. Para uma ordenação descendente use ascending=False.
#### Dica 2: Use o atributo iloc para retornar todas as colunas dos 10 primeiros registros.

In [ ]:
def top10(df, col):
    """retorne os top 10 registros do dataframe df ordenados por col
    """
    ### BEGIN SOLUTION
    return df.sort_values(by=col, ascending=False).iloc[:10,:]
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)


In [ ]:
# Validação
df_pib_pop = merge_pop_pib(path_pop, path_pib)
pib_percapita_2017 = pib_percapita(df_pib_pop)

test_data = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_x': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}
test_data_y = {'cod_ibge7': {3677: '3536505', 3547: '3524709'},
             'municipio_y': {3677: 'Paulínia', 3547: 'Jaguariúna'},
             'uf': {3677: 'SP', 3547: 'SP'},
             'pib_percapita': {3677: 344.8, 3547: 209.3}}

entradas = [[pib_percapita_2017, 'pib_percapita']]
saidas = [test_data]
val1 = validate(top10, entradas, lambda x: x[['cod_ibge7', 'municipio_x', 'uf', 'pib_percapita']].iloc[[0,9]].to_dict(), saidas, "7.3", False)
if not val1:
    saidas = [test_data_y]
    val2 = validate(top10, entradas, lambda x: x[['cod_ibge7', 'municipio_y', 'uf', 'pib_percapita']].iloc[[0,9]].to_dict(), saidas, "7.3", False)
    print(val2)
else:
    print(val1)

### 7.4 Escreva uma função para fazer o merge do dataset de morbidade consolidado com o dataset de população
#### O dataset da população deve ficar a esquerda e o dataset da morbidade a direita no resultado.
#### Dica 1: No dataset de morbidade existem valores NA representados por caracteres específicos em várias colunas do dataframe. Descubra quais são esses caracteres e especifiquei no argumento na_values. Passe nesse argumento uma lista como os caracteres que devem ser interpretados como NA.
#### Dica 2: No dataset de morbidade, especifique o separador, o caracter de números decimais e o tipo do dado do cod_ibge6 como str. Ou seja, dtype={'cod_ibge6': str}.
#### Dica 3: No dataset de população, converta o cod_munic e cod_uf para str no momento de carregar o dataset de população, usando novamente o argumento dtype.
#### Dica 4: No dataset de população, construa o código ibge de 6 dígitos do município a partir das colunas ```cod_uf``` e ```cod_munic```.


In [ ]:
def merge_morbidade_pop(path_pop, path_morbidade):
    """retorne o merge dos datasets de população e morbidade.
    """
    ### BEGIN SOLUTION
    df_pop = pd.read_excel(path_pop, sheet_name="Municipios", dtype={'cod_munic': str, 'cod_uf': str})
    # Concatena o codigo da UF com o codigo do municipio
    df_pop['cod_ibge6'] = df_pop['cod_uf'] + df_pop['cod_munic'].apply(lambda x: x[:-1])
    df_morbidade = pd.read_csv(path_morbidade, sep=';', decimal=',', dtype={'cod_ibge6': str},
                               na_values=['-', '...'])
    return pd.merge(df_pop, df_morbidade, on='cod_ibge6')
    ### END SOLUTION


In [ ]:
## Faça seus testes aqui
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'


In [ ]:
path_morbidade = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/morbidade/consolidado/morbidade.csv'
path_pop = 'https://github.com/alexlopespereira/enapespcd2021/raw/main/data/originais/populacao/estimativa_dou_2017.xlsx'
test_data = {'Internações': 149, 'AIH_aprovadas': 149, 'Valor_total': 101282.26}

entradas = [[path_pop, path_morbidade]]
saidas = [{'Internações': 149, 'AIH_aprovadas': 149, 'Valor_total': 101282.26}]
val1 = validate(merge_morbidade_pop, entradas, lambda x: x[(x['mes']=='Abr') & (x['cod_munic']=='00015')].loc[:,['Internações', 'AIH_aprovadas', 'Valor_total']].to_dict(orient='records')[0], saidas, "7.4", False)
if not val1:
    val2 = validate(top10, entradas, lambda x: x.iloc[0, 8:11].to_dict(), saidas, "7.4", False)
    print(val2)
else:
    print(val1)